# Capstone Project - The Battle of Neighborhoods ( Week 1 & 2 )¶


### Background¶


According to Bloomberg News, the London Housing Market is in a rut. It is now facing a number of different headwinds, including the prospect of higher taxes and a warning from the Bank of England that U.K. home values could fall as much as 30 percent in the event of a disorderly exit from the European Union. More specifically, four overlooked cracks suggest that the London market may be in worse shape than many realize: hidden price falls, record-low sales, homebuilder exodus and tax hikes addressing overseas buyers of homes in England and Wales.

### Business Problem¶


In this scenario, it is urgent to adopt machine learning tools in order to assist homebuyers clientele in London to make wise and effective decisions. As a result, the business problem we are currently posing is: how could we provide support to homebuyers clientele in to purchase a suitable real estate in London in this uncertain economic and financial scenario?

To solve this business problem, we are going to cluster London neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We will recommend profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

### Data section¶


Data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). The following fields comprise the address data included in Price Paid Data: Postcode; PAON Primary Addressable Object Name. Typically the house number or name; SAON Secondary Addressable Object Name. If there is a sub-building, for example, the building is divided into flats, there will be a SAON; Street; Locality; Town/City; District; County.

To explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we will access data through FourSquare API interface and arrange them as a dataframe for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we will be able to recommend profitable real estate investments.

###  Methodology section¶


The Methodology section will describe the main components of our analysis and predication system. The Methodology section comprises four stages:

1. Collect Inspection Data
2. Explore and Understand Data
3. Data preparation and preprocessing 
4. Modeling

#### 1. Collect Inspection Data¶


After importing the necessary libraries, we download the data from the HM Land Registry website as follows:



In [1]:
import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
#Read the data for examination (Source: http://landregistry.data.gov.uk/)
df_ppd = pd.read_csv("http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018.csv")

Before using data, we will have to explore and understand it.



#### 2. Explore and Understand Data¶


We read the dataset that we collected from the HM Land Registry website into a pandas' data frame and display the first five rows of it as follows:

In [3]:
df_ppd.head(5)


,{79A74E21-C934-1289-E053-6B04A8C01627},177000,2018-09-21 00:00,LE4 6EE,S,N,F,201,Unnamed: 8,BELPER STREET,Unnamed: 10,LEICESTER,LEICESTER.1,LEICESTER.2,A,A.1
0,{79A74E21-C935-1289-E053-6B04A8C01627},90000,2018-10-01 00:00,LE18 2AE,F,N,L,27,NaN,ELIZABETH COURT,NaN,WIGSTON,OADBY AND WIGSTON,LEICESTERSHIRE,A,A
1,{79A74E21-C936-1289-E053-6B04A8C01627},375000,2018-10-04 00:00,LE11 3HG,D,N,F,6,NaN,GOLDFINCH CLOSE,NaN,LOUGHBOROUGH,CHARNWOOD,LEICESTERSHIRE,A,A
2,{79A74E21-C937-1289-E053-6B04A8C01627},142500,2018-10-08 00:00,LE3 6UY,S,N,F,19,NaN,PINEHURST CLOSE,NaN,LEICESTER,LEICESTER,LEICESTER,A,A
3,{79A74E21-C938-1289-E053-6B04A8C01627},157500,2018-10-22 00:00,LE13 0JH,S,N,F,103,NaN,WEST AVENUE,NaN,MELTON MOWBRAY,MELTON,LEICESTERSHIRE,A,A
4,{79A74E21-C939-1289-E053-6B04A8C01627},192500,2018-10-02 00:00,LE12 7UT,T,N,F,26,NaN,MELODY DRIVE,SILEBY,LOUGHBOROUGH,CHARNWOOD,LEICESTERSHIRE,A,A


In [4]:
df_ppd.shape


(1017356, 16)

Our dataset consists of over 700000 rows and 16 columns. We will now prepare and preprocess data accordingly.



#### 3. Data preparation and preprocessing¶


At this stage, we prepare our dataset for the modeling process, opting for the most suitable machine learning algorithm for our scope. Accordingly, we perform the following steps:

* Rename the column names
* Format the date column
* Sort data by date of sale
* Select data only for the city of London
* Make a list of street names in London
* Calculate the street-wise average price of the property
* Read the street-wise coordinates into a data frame, eliminating recurring word London from individual names
* Join the data to find the coordinates of locations which fit into client's budget
* Plot recommended locations on London map along with current market prices

In [5]:
# Assign meaningful column names
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

# Format the date column
df_ppd['Date_Transfer'] = df_ppd['Date_Transfer'].apply(pd.to_datetime)

# Delete all obsolete transactions which were done before 2016
df_ppd.drop(df_ppd[df_ppd.Date_Transfer.dt.year < 2016].index, inplace=True)

# Sort by Date of Sale
df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

df_ppd_london = df_ppd.query("Town_City == 'LONDON'")

# Make a list of street names in LONDON
streets = df_ppd_london['Street'].unique().tolist()

df_grp_price = df_ppd_london.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
df_grp_price.columns = ['Street', 'Avg_Price']

#Input your Budget's Upper Limit and Lower Limit - Find the locations df_grp_price which fits your budget
df_affordable = df_grp_price.query("(Avg_Price >= 2200000) & (Avg_Price <= 2500000)")

# Display the dataframe
df_affordable

,Street,Avg_Price
195,ALBION SQUARE,2.450000e+06
390,ANHALT ROAD,2.435000e+06
405,ANSDELL TERRACE,2.250000e+06
420,APPLEGARTH ROAD,2.400000e+06
698,AYLESTONE AVENUE,2.286667e+06
851,BARONSMEAD ROAD,2.375000e+06
975,BEAUCLERC ROAD,2.480000e+06
1096,BELVEDERE DRIVE,2.340000e+06
1209,BICKENHALL STREET,2.208500e+06
1247,BIRCHLANDS AVENUE,2.217000e+06


In [6]:
import pandas as pd
import numpy as np
import datetime as DT
import hmac
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
# import k-means from clustering stage
from sklearn.cluster import KMeans

for index, item in df_affordable.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.Street}")

index: 195
item: Street       ALBION SQUARE
Avg_Price         2.45e+06
Name: 195, dtype: object
item.Street only: ALBION SQUARE
index: 390
item: Street       ANHALT ROAD
Avg_Price      2.435e+06
Name: 390, dtype: object
item.Street only: ANHALT ROAD
index: 405
item: Street       ANSDELL TERRACE
Avg_Price           2.25e+06
Name: 405, dtype: object
item.Street only: ANSDELL TERRACE
index: 420
item: Street       APPLEGARTH ROAD
Avg_Price            2.4e+06
Name: 420, dtype: object
item.Street only: APPLEGARTH ROAD
index: 698
item: Street       AYLESTONE AVENUE
Avg_Price         2.28667e+06
Name: 698, dtype: object
item.Street only: AYLESTONE AVENUE
index: 851
item: Street       BARONSMEAD ROAD
Avg_Price          2.375e+06
Name: 851, dtype: object
item.Street only: BARONSMEAD ROAD
index: 975
item: Street       BEAUCLERC ROAD
Avg_Price          2.48e+06
Name: 975, dtype: object
item.Street only: BEAUCLERC ROAD
index: 1096
item: Street       BELVEDERE DRIVE
Avg_Price           2.34e+06
Name

In [7]:
geolocator = Nominatim()


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  if __name__ == '__main__':


In [8]:
df_affordable['city_coord'] = df_affordable['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))
df_affordable

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Street,Avg_Price,city_coord
195,ALBION SQUARE,2.450000e+06,"(-41.27375755, 173.289393239104)"
390,ANHALT ROAD,2.435000e+06,"(51.4803265, -0.1667607)"
405,ANSDELL TERRACE,2.250000e+06,"(51.4998899, -0.1891027)"
420,APPLEGARTH ROAD,2.400000e+06,"(53.749244, -0.32678)"
698,AYLESTONE AVENUE,2.286667e+06,"(51.5409157, -0.2178742)"
851,BARONSMEAD ROAD,2.375000e+06,"(51.4773147, -0.239457)"
975,BEAUCLERC ROAD,2.480000e+06,"(51.4995771, -0.2290331)"
1096,BELVEDERE DRIVE,2.340000e+06,"(51.4249173, -0.2120774)"
1209,BICKENHALL STREET,2.208500e+06,"(51.5211969, -0.1589341)"
1247,BIRCHLANDS AVENUE,2.217000e+06,"(51.4483941, -0.1604676)"


In [9]:
df_affordable[['Latitude', 'Longitude']] = df_affordable['city_coord'].apply(pd.Series)
df_affordable

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,Street,Avg_Price,city_coord,Latitude,Longitude
195,ALBION SQUARE,2.450000e+06,"(-41.27375755, 173.289393239104)",-41.273758,173.289393
390,ANHALT ROAD,2.435000e+06,"(51.4803265, -0.1667607)",51.480326,-0.166761
405,ANSDELL TERRACE,2.250000e+06,"(51.4998899, -0.1891027)",51.499890,-0.189103
420,APPLEGARTH ROAD,2.400000e+06,"(53.749244, -0.32678)",53.749244,-0.326780
698,AYLESTONE AVENUE,2.286667e+06,"(51.5409157, -0.2178742)",51.540916,-0.217874
851,BARONSMEAD ROAD,2.375000e+06,"(51.4773147, -0.239457)",51.477315,-0.239457
975,BEAUCLERC ROAD,2.480000e+06,"(51.4995771, -0.2290331)",51.499577,-0.229033
1096,BELVEDERE DRIVE,2.340000e+06,"(51.4249173, -0.2120774)",51.424917,-0.212077
1209,BICKENHALL STREET,2.208500e+06,"(51.5211969, -0.1589341)",51.521197,-0.158934
1247,BIRCHLANDS AVENUE,2.217000e+06,"(51.4483941, -0.1604676)",51.448394,-0.160468


In [10]:
df = df_affordable.drop(columns=['city_coord'])
df

,Street,Avg_Price,Latitude,Longitude
195,ALBION SQUARE,2.450000e+06,-41.273758,173.289393
390,ANHALT ROAD,2.435000e+06,51.480326,-0.166761
405,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103
420,APPLEGARTH ROAD,2.400000e+06,53.749244,-0.326780
698,AYLESTONE AVENUE,2.286667e+06,51.540916,-0.217874
851,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457
975,BEAUCLERC ROAD,2.480000e+06,51.499577,-0.229033
1096,BELVEDERE DRIVE,2.340000e+06,51.424917,-0.212077
1209,BICKENHALL STREET,2.208500e+06,51.521197,-0.158934
1247,BIRCHLANDS AVENUE,2.217000e+06,51.448394,-0.160468


In [11]:
address = 'London, UK'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of London City are 51.5073219, -0.1276474.


In [12]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df['Latitude'], df['Longitude'], df['Avg_Price'], df['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [13]:
#Define Foursquare Credentials and Version

CLIENT_ID = 'S3ONPSOKENADL51USBUU4BATDOAQZUZ0WJL13VCXOPDXKQ33' # Foursquare ID
CLIENT_SECRET = '4HGT1KWEEAAPJYSWGNN0IGBOYHVVOMA01AAIH5Q5QSJXHH4T' # Foursquare Secret
VERSION = '20181206' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: S3ONPSOKENADL51USBUU4BATDOAQZUZ0WJL13VCXOPDXKQ33
CLIENT_SECRET:4HGT1KWEEAAPJYSWGNN0IGBOYHVVOMA01AAIH5Q5QSJXHH4T


We can now proceed to the Modeling phase. We will analyze neighborhoods to recommend real estates where home buyers can make a real estate investment. We will then recommend profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

#### 4. Modeling¶


After exploring the dataset and gaining insights into it, we are ready to use the clustering methodology to analyze real estates. We will use the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


# Run the above function on each location and create a new dataframe called location_venues and display it.
location_venues = getNearbyVenues(names=df['Street'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

ALBION SQUARE
ANHALT ROAD
ANSDELL TERRACE
APPLEGARTH ROAD
AYLESTONE AVENUE
BARONSMEAD ROAD
BEAUCLERC ROAD
BELVEDERE DRIVE
BICKENHALL STREET
BIRCHLANDS AVENUE
BRAMPTON GROVE
BRIARDALE GARDENS
BROOKWAY
BURBAGE ROAD
BURY WALK
CALLCOTT STREET
CAMPDEN HILL ROAD
CAMPION ROAD
CANNING PLACE
CARLISLE ROAD
CARLTON GARDENS
CARLYLE COURT
CHALCOT SQUARE
CHARLES LANE
CHELSEA CRESCENT
CHESTER CLOSE NORTH
CHEYNE COURT
CHEYNE ROW
CHISWICK MALL
CLARENDON STREET
CLONCURRY STREET
COLBECK MEWS
CORNWALL TERRACE MEWS
COURT LANE GARDENS
CRESCENT GROVE
DALEBURY ROAD
DEWHURST ROAD
DORIA ROAD
DOWNSHIRE HILL
DUCHESS WALK
ECCLESTON SQUARE MEWS
EGBERT STREET
EGERTON PLACE
ELM PARK ROAD
FRANK DIXON WAY
FULTON MEWS
GERARD ROAD
GERRARD ROAD
GIRDLERS ROAD
GLOUCESTER CRESCENT
GORDON PLACE
GRAFTON SQUARE
GRAHAM TERRACE
HARMAN DRIVE
HAVANNAH STREET
HAZLEWELL ROAD
HEREFORD MEWS
HERONDALE AVENUE
HIGHGATE HIGH STREET
HIGHWOOD HILL
HILLGATE PLACE
HOLLYCROFT AVENUE
HOLLYWOOD MEWS
HONEYWELL ROAD
HORTENSIA ROAD
HOXTON SQUARE
HUN

In [15]:
location_venues


,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ALBION SQUARE,-41.273758,173.289393,The Free House,-41.273340,173.287364,Bar
1,ALBION SQUARE,-41.273758,173.289393,The Indian Cafe,-41.273308,173.286530,Indian Restaurant
2,ALBION SQUARE,-41.273758,173.289393,The Bridge Street Collective,-41.272520,173.285517,Café
3,ALBION SQUARE,-41.273758,173.289393,Queen's Gardens,-41.273671,173.291383,Park
4,ALBION SQUARE,-41.273758,173.289393,Urban,-41.274355,173.286317,New American Restaurant
5,ALBION SQUARE,-41.273758,173.289393,Deville Cafe,-41.271941,173.285535,Beer Garden
6,ALBION SQUARE,-41.273758,173.289393,Burger Culture,-41.274750,173.284030,Burger Joint
7,ALBION SQUARE,-41.273758,173.289393,Fish Stop,-41.276010,173.289592,Fish & Chips Shop
8,ALBION SQUARE,-41.273758,173.289393,Fresh Choice,-41.272194,173.287218,Supermarket
9,ALBION SQUARE,-41.273758,173.289393,Mango,-41.274460,173.285345,Indian Restaurant


In [16]:
location_venues.groupby('Street').count()


,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
ALBION SQUARE,26,26,26,26,26,26
ANHALT ROAD,15,15,15,15,15,15
ANSDELL TERRACE,58,58,58,58,58,58
APPLEGARTH ROAD,4,4,4,4,4,4
AYLESTONE AVENUE,4,4,4,4,4,4
BARONSMEAD ROAD,12,12,12,12,12,12
BEAUCLERC ROAD,29,29,29,29,29,29
BELVEDERE DRIVE,40,40,40,40,40,40
BICKENHALL STREET,100,100,100,100,100,100


In [17]:
# get the List of Unique Categories
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))

There are 343 uniques categories.


In [18]:
location_venues.shape


(5846, 7)

In [19]:
# one hot encoding
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Street'] = location_venues['Street'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Street,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,...,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo Exhibit
0,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
london_grouped = venues_onehot.groupby('Street').mean().reset_index()
london_grouped

,Street,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,...,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo Exhibit
0,ALBION SQUARE,0.000000,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.038462,...,0.000000,0.0,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.0
1,ANHALT ROAD,0.000000,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.0
2,ANSDELL TERRACE,0.000000,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.017241,...,0.000000,0.0,0.000000,0.000,0.00,0.000000,0.000000,0.017241,0.000000,0.0
3,APPLEGARTH ROAD,0.000000,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.0
4,AYLESTONE AVENUE,0.000000,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.0
5,BARONSMEAD ROAD,0.000000,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.0
6,BEAUCLERC ROAD,0.000000,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.0
7,BELVEDERE DRIVE,0.000000,0.0,0.0,0.000000,0.000000,0.000,0.0,0.025000,0.000000,...,0.025000,0.0,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.0
8,BICKENHALL STREET,0.000000,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000,0.00,0.010000,0.000000,0.010000,0.010000,0.0
9,BIRCHLANDS AVENUE,0.000000,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.0


In [21]:
london_grouped.shape


(149, 344)

In [22]:
# What are the top 5 venues/facilities nearby profitable real estate investments?#

num_top_venues = 5

for hood in london_grouped['Street']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALBION SQUARE----
               venue  freq
0               Café  0.19
1         Restaurant  0.08
2  Indian Restaurant  0.08
3                Pub  0.08
4        Coffee Shop  0.08


----ANHALT ROAD----
                  venue  freq
0                   Pub  0.27
1         Grocery Store  0.13
2    Chinese Restaurant  0.07
3  Gym / Fitness Center  0.07
4          Cocktail Bar  0.07


----ANSDELL TERRACE----
                venue  freq
0  Italian Restaurant  0.07
1      Clothing Store  0.07
2          Restaurant  0.07
3               Hotel  0.05
4           Juice Bar  0.05


----APPLEGARTH ROAD----
               venue  freq
0                Pub  0.50
1          Nightclub  0.25
2             Casino  0.25
3  Accessories Store  0.00
4             Palace  0.00


----AYLESTONE AVENUE----
               venue  freq
0               Park  0.50
1           Bus Stop  0.25
2      Movie Theater  0.25
3  Accessories Store  0.00
4             Palace  0.00


----BARONSMEAD ROAD----
               ve

In [23]:
# Define a function to return the most common venues/facilities nearby real estate investments#

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

        
# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = london_grouped['Street']

for ind in np.arange(london_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

    
venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALBION SQUARE,Café,Pub,Bar,Indian Restaurant,Coffee Shop,Restaurant,New American Restaurant,French Restaurant,Seafood Restaurant,Fish & Chips Shop
1,ANHALT ROAD,Pub,Grocery Store,Japanese Restaurant,Garden,Gym / Fitness Center,Diner,Chinese Restaurant,Plaza,English Restaurant,Cocktail Bar
2,ANSDELL TERRACE,Restaurant,Italian Restaurant,Clothing Store,Pub,Juice Bar,Hotel,French Restaurant,Indian Restaurant,Chinese Restaurant,Café
3,APPLEGARTH ROAD,Pub,Nightclub,Casino,Food Court,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food
4,AYLESTONE AVENUE,Park,Movie Theater,Bus Stop,Food Stand,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop


In [24]:
venues_sorted.shape


(149, 11)

In [25]:
london_grouped.shape


(149, 344)

In [26]:
london_grouped=df


After our inspection of venues/facilities/amenities nearby the most profitable real estate investments in London, we could begin by clustering properties by venues/facilities/amenities nearby.

In [27]:
#Distribute in 5 Clusters

# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([1, 2, 0, 2, 3, 4, 1, 4, 0, 0, 1, 2, 2, 1, 1, 4, 4, 1, 2, 0, 1, 3,
       3, 2, 1, 1, 0, 2, 3, 0, 2, 4, 4, 4, 3, 2, 2, 4, 0, 1, 4, 3, 0, 3,
       0, 3, 0, 0, 2, 4], dtype=int32)

In [28]:
#Dataframe to include Clusters

london_grouped_clustering=df
london_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude
195,ALBION SQUARE,2.450000e+06,-41.273758,173.289393
390,ANHALT ROAD,2.435000e+06,51.480326,-0.166761
405,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103
420,APPLEGARTH ROAD,2.400000e+06,53.749244,-0.326780
698,AYLESTONE AVENUE,2.286667e+06,51.540916,-0.217874


In [29]:
london_grouped_clustering.shape


(156, 4)

In [30]:
df.shape


(156, 4)

In [31]:
london_grouped_clustering.dtypes


Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [32]:
df.dtypes


Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [33]:
# add clustering labels
london_grouped_clustering['Cluster Labels'] = kmeans.labels_

# merge london_grouped with london_data to add latitude/longitude for each neighborhood
london_grouped_clustering = london_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')

london_grouped_clustering.head(30) # check the last columns!

,Street,Avg_Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
195,ALBION SQUARE,2.450000e+06,-41.273758,173.289393,1,Café,Pub,Bar,Indian Restaurant,Coffee Shop,Restaurant,New American Restaurant,French Restaurant,Seafood Restaurant,Fish & Chips Shop
390,ANHALT ROAD,2.435000e+06,51.480326,-0.166761,2,Pub,Grocery Store,Japanese Restaurant,Garden,Gym / Fitness Center,Diner,Chinese Restaurant,Plaza,English Restaurant,Cocktail Bar
405,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103,0,Restaurant,Italian Restaurant,Clothing Store,Pub,Juice Bar,Hotel,French Restaurant,Indian Restaurant,Chinese Restaurant,Café
420,APPLEGARTH ROAD,2.400000e+06,53.749244,-0.326780,2,Pub,Nightclub,Casino,Food Court,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food
698,AYLESTONE AVENUE,2.286667e+06,51.540916,-0.217874,3,Park,Movie Theater,Bus Stop,Food Stand,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop
851,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457,4,Nature Preserve,Breakfast Spot,Park,Coffee Shop,Food & Drink Shop,Pizza Place,Café,Movie Theater,Restaurant,Pub
975,BEAUCLERC ROAD,2.480000e+06,51.499577,-0.229033,1,Pub,Coffee Shop,Hotel,Chinese Restaurant,Thai Restaurant,Grocery Store,Falafel Restaurant,Fish & Chips Shop,Street Food Gathering,Cocktail Bar
1096,BELVEDERE DRIVE,2.340000e+06,51.424917,-0.212077,4,Pub,Coffee Shop,French Restaurant,Bakery,Italian Restaurant,Thai Restaurant,Gym / Fitness Center,Lounge,Café,Clothing Store
1209,BICKENHALL STREET,2.208500e+06,51.521197,-0.158934,0,Hotel,Restaurant,Coffee Shop,Gastropub,Café,Pub,Chinese Restaurant,Pizza Place,Indian Restaurant,Sandwich Place
1247,BIRCHLANDS AVENUE,2.217000e+06,51.448394,-0.160468,0,Pub,French Restaurant,Brewery,Breakfast Spot,Bakery,Lake,Chinese Restaurant,Coffee Shop,Train Station,Zoo Exhibit


In [34]:
 Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_grouped_clustering['Latitude'], london_grouped_clustering['Longitude'], london_grouped_clustering['Street'], london_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

SyntaxError: invalid syntax (<ipython-input-34-560a617d874b>, line 1)

In [ ]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 0, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

In [ ]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 1, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

In [ ]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 2, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

In [ ]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 3, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

In [ ]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 4, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

### Results and Discussion section¶


First of all, even though the London Housing Market may be in a rut, it is still an "ever-green" for business affairs.

We may discuss our results under two main perspectives.

First, we may examine them according to neighborhoods/London areas. It is interesting to note that, although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair.

Second, we may analyze our results according to the five clusters we have produced. Even though, all clusters could praise an optimal range of facilities and amenities, we have found two main patterns. The first pattern we are referring to, i.e. Clusters 0, 2 and 4, may target home buyers prone to live in 'green' areas with parks, waterfronts. Instead, the second pattern we are referring to, i.e. Clusters 1 and 3, may target individuals who love pubs, theatres and soccer.

### Conclusion

To sum up, according to Bloomberg News, the London Housing Market is in a rut. It is now facing a number of different headwinds, including the prospect of higher taxes and a warning from the Bank of England that U.K. home values could fall as much as 30 percent in the event of a disorderly exit from the European Union. In this scenario, it is urgent to adopt machine learning tools in order to assist homebuyers clientele in London to make wise and effective decisions. As a result, the business problem we were posing was: how could we provide support to homebuyers clientele in to purchase a suitable real estate in London in this uncertain economic and financial scenario?

To solve this business problem, we clustered London neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We recommended profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

First, we gathered data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). Moreover, to explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we accessed data through FourSquare API interface and arranged them as a data frame for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we were able to recommend profitable real estate investments.

Second, The Methodology section comprised four stages: 1. Collect Inspection Data; 2. Explore and Understand Data; 3. Data preparation and preprocessing; 4. Modeling. In particular, in the modeling section, we used the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.

Finally, we drew the conclusion that even though the London Housing Market may be in a rut, it is still an "ever-green" for business affairs. We discussed our results under two main perspectives. First, we examined them according to neighborhoods/London areas. although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair. Second, we analyzed our results according to the five clusters we produced. While Clusters 0, 2 and 4 may target home buyers prone to live in 'green' areas with parks, waterfronts, Clusters 1 and 3 may target individuals who love pubs, theatres and soccer.